In [1]:
pip list


Package                     Version
--------------------------- --------
absl-py                     1.4.0
anyio                       3.7.0
appdirs                     1.4.4
appnope                     0.1.3
argon2-cffi                 21.3.0
argon2-cffi-bindings        21.2.0
arrow                       1.2.3
asttokens                   2.2.1
astunparse                  1.6.3
attrs                       23.1.0
backcall                    0.2.0
beautifulsoup4              4.12.2
bleach                      6.0.0
brotlipy                    0.7.0
cachetools                  5.3.1
catboost                    1.2
certifi                     2023.5.7
cffi                        1.15.1
chardet                     5.1.0
charset-normalizer          2.0.4
combo                       0.1.3
comm                        0.1.3
contourpy                   1.1.0
cryptography                39.0.1
cycler                      0.11.0
debugpy                     1.6.7
decorator                   5.1.1
d

In [5]:
#未加 from skimage import feature as ft
import os
import cv2
from  tqdm  import  tqdm 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
import random

from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier
import xgboost
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [6]:
# 图像均衡化
def equalize(image):
    # 分割B,G,R （cv2读取图像的格式即为[B,G,R]，与matplotlib的[R,G,B]不同）
    b,g,r = cv2.split(image)
    # 依次均衡化
    b = cv2.equalizeHist(b)
    g = cv2.equalizeHist(g)
    r = cv2.equalizeHist(r)
    # 结合成一个图像
    equ_img = cv2.merge((b,g,r))
    
    return equ_img


In [8]:
#以1_Pre_test-h02060.jpg为例
image=cv2.imread('h02060.jpg')

In [9]:
equalize(image)

array([[[187, 187, 187],
        [187, 187, 187],
        [187, 187, 187],
        ...,
        [211, 211, 211],
        [211, 211, 211],
        [211, 211, 211]],

       [[211, 211, 211],
        [211, 211, 211],
        [211, 211, 211],
        ...,
        [211, 211, 211],
        [211, 211, 211],
        [211, 211, 211]],

       [[233, 233, 233],
        [233, 233, 233],
        [233, 233, 233],
        ...,
        [211, 211, 211],
        [211, 211, 211],
        [211, 211, 211]],

       ...,

       [[211, 211, 211],
        [223, 223, 223],
        [233, 233, 233],
        ...,
        [211, 211, 211],
        [223, 223, 223],
        [223, 223, 223]],

       [[211, 211, 211],
        [223, 223, 223],
        [233, 233, 233],
        ...,
        [211, 211, 211],
        [223, 223, 223],
        [223, 223, 223]],

       [[211, 211, 211],
        [223, 223, 223],
        [233, 233, 233],
        ...,
        [211, 211, 211],
        [223, 223, 223],
        [223, 223, 223]]

In [14]:
# 提取图片中绿色（叶子）的部分 ✖️  二值化、过滤滤波
def extractGreen(image):
    # 绿色范围
    lower_green = np.array([0, 0, 0], dtype="uint8")  # 颜色下限
    upper_green = np.array([255, 255, 255], dtype="uint8")  # 颜色上限
    
    # 高斯滤波
    img_blur = cv2.GaussianBlur(image, (3, 3), 0)
    img_blur = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)
    # 根据阈值找到对应颜色，二值化
    mask = cv2.inRange(img_blur, lower_green, upper_green)
    
    # 掩膜函数
    output = cv2.bitwise_and(image, image, mask=mask)

    return output


In [15]:
extractGreen(image)

array([[[245, 245, 245],
        [245, 245, 245],
        [245, 245, 245],
        ...,
        [247, 247, 247],
        [247, 247, 247],
        [247, 247, 247]],

       [[247, 247, 247],
        [247, 247, 247],
        [247, 247, 247],
        ...,
        [247, 247, 247],
        [247, 247, 247],
        [247, 247, 247]],

       [[249, 249, 249],
        [249, 249, 249],
        [249, 249, 249],
        ...,
        [247, 247, 247],
        [247, 247, 247],
        [247, 247, 247]],

       ...,

       [[247, 247, 247],
        [248, 248, 248],
        [249, 249, 249],
        ...,
        [247, 247, 247],
        [248, 248, 248],
        [248, 248, 248]],

       [[247, 247, 247],
        [248, 248, 248],
        [249, 249, 249],
        ...,
        [247, 247, 247],
        [248, 248, 248],
        [248, 248, 248]],

       [[247, 247, 247],
        [248, 248, 248],
        [249, 249, 249],
        ...,
        [247, 247, 247],
        [248, 248, 248],
        [248, 248, 248]]

In [16]:
# 提取图像的SIFT特征
def sift_feature(image_list):
    feature_sift_list = []  # SIFT特征向量列表
    
    sift = cv2.xfeatures2d.SIFT_create()
    
    for i in tqdm(range(len(image_list))):
        # 转为灰度图
        image = cv2.cvtColor(image_list[i], cv2.COLOR_BGR2GRAY)
        # 获取SIFT特征，kp为关键点信息，des为关键点特征矩阵形式
        kp, des = sift.detectAndCompute(image, None)
        feature_sift_list.append(des)
        
    return feature_sift_list
